In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/P.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MSHOT","6MSHOT","9MSHOT","WINGSHOT","PIVOTSHOT","FASTBREAKSHOT"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsacc.csv', index=False)


Epoch 1/100
300/300 [==============================] - 1s 2ms/step - loss: 2.3051 - mean_absolute_error: 1.2109 - val_loss: 1.9934 - val_mean_absolute_error: 1.1493
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0242 - mean_absolute_error: 1.1261 - val_loss: 1.7549 - val_mean_absolute_error: 1.0668
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7790 - mean_absolute_error: 1.0431 - val_loss: 1.5344 - val_mean_absolute_error: 0.9710
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5564 - mean_absolute_error: 0.9545 - val_loss: 1.3525 - val_mean_absolute_error: 0.9096
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.3842 - mean_absolute_error: 0.8869 - val_loss: 1.2135 - val_mean_absolute_error: 0.8252
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.2505 - mean_absolute_error: 0.8220 - val_loss: 1.1131 - val_mean_absolute_error: 0.7575
Epoch 7/10

C:\Users\Wandi\AppData\Local\Temp\ipykernel_148096\1713343205.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 2.8470 - mean_absolute_error: 1.2223 - val_loss: 2.1923 - val_mean_absolute_error: 1.2064
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.1982 - mean_absolute_error: 1.1909 - val_loss: 1.9397 - val_mean_absolute_error: 1.1302
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 1.9539 - mean_absolute_error: 1.1139 - val_loss: 1.7179 - val_mean_absolute_error: 1.0521
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7355 - mean_absolute_error: 1.0312 - val_loss: 1.5306 - val_mean_absolute_error: 0.9748
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5518 - mean_absolute_error: 0.9607 - val_loss: 1.3679 - val_mean_absolute_error: 0.8981
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.3874 - mean_absolute_error: 0.8987 - val_loss: 1.2291 - val_mean_absolute_error: 0.8195
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_148096\1713343205.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 3.9064 - mean_absolute_error: 1.5490 - val_loss: 2.9479 - val_mean_absolute_error: 1.3902
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.7043 - mean_absolute_error: 1.3323 - val_loss: 2.2070 - val_mean_absolute_error: 1.2201
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0603 - mean_absolute_error: 1.1546 - val_loss: 1.7349 - val_mean_absolute_error: 1.0832
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7481 - mean_absolute_error: 1.0551 - val_loss: 1.5544 - val_mean_absolute_error: 1.0030
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5948 - mean_absolute_error: 0.9956 - val_loss: 1.4184 - val_mean_absolute_error: 0.9496
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.4549 - mean_absolute_error: 0.9476 - val_loss: 1.2991 - val_mean_absolute_error: 0.8947
Epoch 7/100
300/300 [=

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_148096\1713343205.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 3.3211 - mean_absolute_error: 1.3756 - val_loss: 2.5810 - val_mean_absolute_error: 1.3007
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.5279 - mean_absolute_error: 1.2957 - val_loss: 2.2941 - val_mean_absolute_error: 1.2679
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.2236 - mean_absolute_error: 1.2275 - val_loss: 1.9844 - val_mean_absolute_error: 1.1703
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.9714 - mean_absolute_error: 1.1454 - val_loss: 1.7898 - val_mean_absolute_error: 1.0963
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7952 - mean_absolute_error: 1.0872 - val_loss: 1.6328 - val_mean_absolute_error: 1.0334
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.6478 - mean_absolute_error: 1.0263 - val_loss: 1.5004 - val_mean_absolute_error: 0.9933
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_148096\1713343205.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


300/300 [==============================] - 1s 2ms/step - loss: 3.1272 - mean_absolute_error: 1.4148 - val_loss: 2.6600 - val_mean_absolute_error: 1.3301
Epoch 2/100
300/300 [==============================] - 0s 1ms/step - loss: 2.5023 - mean_absolute_error: 1.2837 - val_loss: 2.1814 - val_mean_absolute_error: 1.2220
Epoch 3/100
300/300 [==============================] - 0s 1ms/step - loss: 2.0524 - mean_absolute_error: 1.1733 - val_loss: 1.7869 - val_mean_absolute_error: 1.0922
Epoch 4/100
300/300 [==============================] - 0s 1ms/step - loss: 1.7552 - mean_absolute_error: 1.0729 - val_loss: 1.5943 - val_mean_absolute_error: 1.0257
Epoch 5/100
300/300 [==============================] - 0s 1ms/step - loss: 1.5931 - mean_absolute_error: 1.0115 - val_loss: 1.4499 - val_mean_absolute_error: 0.9614
Epoch 6/100
300/300 [==============================] - 0s 1ms/step - loss: 1.4531 - mean_absolute_error: 0.9530 - val_loss: 1.3279 - val_mean_absolute_error: 0.9054
Epoch 7/100
300/300 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_148096\1713343205.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
